In [1]:
import numpy as np
from matplotlib import pyplot as plt
from astropy.table import Table
import os
import sys
sys.path.append('../')
import utils as ul
import copy
import pickle
import gzip
from scipy.signal import medfilt
import h5py
from scipy.stats import norm
import time
import pymultinest
import json
import copy
font = {'family':'serif', 'size':14}
plt.rc('font', **font)
plt.rc('xtick', labelsize='small')
plt.rc('ytick', labelsize='small')
plt.rc('axes', labelsize='medium')

dir0 = os.getcwd()

in_dir = '/data/inspur_disk03/userdir/caoxy/eboss_lya/data/'
lya_hits = pickle.load(gzip.open(in_dir+'lya_hits_is_oii.pzip'))


dir1 = os.getcwd()
os.chdir('/data/inspur_disk03/userdir/caoxy/eboss_lya/data/2')
dir2 = os.getcwd()

plate_hits = lya_hits['plate_hits']
fiber_hits = lya_hits['fiber_hits']
mjd_hits = lya_hits['mjd_hits']
spec_id_hits = lya_hits['spec_id']
peak_id_hits = lya_hits['peak_id']
peak_wave_hits = lya_hits['peak_wave']
peak_sn_hits = lya_hits['peak_sn']


#os.chdir(dir0)
for ii in range(plate_hits.size): #enumerate(plate_hits[3:4]):
    this_plate = plate_hits[ii]
    this_mjd = mjd_hits[ii]
    this_fiber = fiber_hits[ii]
    this_spec_id = spec_id_hits[ii]
    this_peak_id = peak_id_hits[ii]
    this_peak_wave = peak_wave_hits[ii]
    this_peak_sn = peak_sn_hits[ii]
    h5_file = '{}_{}.h5'.format(this_plate,this_mjd)
    with h5py.File(h5_file, 'r') as f:
        arr0 = f['flux']
        flux = arr0[:]
        
        arr1 = f['loglam']
        loglam = arr1[:]
        
        arr2 = f['zans']
        zans = arr2[:]
        
        arr3 = f['newsynflux']
        synflux = arr3[:]
        
        arr4 = f['res_ivar']
        res_ivar = arr4[:]
    

    this_wave = 10**loglam
    this_flux = flux[this_spec_id,:]
    this_synflux = synflux[this_spec_id,:]
    this_resflux = this_flux - this_synflux
    this_res_ivar = res_ivar[this_spec_id,:]
    this_sn = np.sqrt(this_resflux**2 * this_res_ivar)   
    
    hb_lam = 4862.683
    o3_lam = 5008.239
    ha_lam = 6564.614
    o2_lam = np.sqrt(3727.092 * 3729.875)
    z_fake = this_peak_wave/o2_lam
    fake_lam_list = np.array([hb_lam, o3_lam, ha_lam])*z_fake
    
    hw=20
    fig_f = 2
    fig,ax = plt.subplots(figsize=(3*fig_f,2*fig_f))
    ax.plot(this_wave,this_flux,linewidth=1.0)
    ax.axvspan(this_wave[this_peak_id-hw], this_wave[this_peak_id+hw+1], facecolor='g', alpha=0.5)
    
    for lam_item in fake_lam_list:
        ax.axvspan(lam_item-hw, lam_item+hw, facecolor='cyan', alpha=0.5)
    
    textstr = r'Spec-ID: {}-{}-{}'.format(this_plate,this_mjd,this_fiber)
    ax.text(0.35, 0.95, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment='top')
    ax.minorticks_on()
    ax.set_xlim(3700,9000)
    id_tmp = np.where(np.logical_and(this_wave<4800,this_wave>3700))
    up_lim = this_flux[id_tmp].max()*1.2
    if this_flux[id_tmp].min() < 0:
        low_lim = this_flux[id_tmp].min()*1.2
    else:
        low_lim = -0.1
    ax.set_ylim(low_lim,up_lim)
    ax.set_ylabel('$\\rm f_{\\lambda} \ [10^{-17} erg cm^{-2} s^{-1} Ang^{-1}]$') #,fontsize=20
    ax.set_xlabel(r'Observed wavelength [Ang]')
    fig.savefig('{}/oii_spectra/{}_{}_{}.pdf'.format(dir0,this_plate,this_mjd,this_fiber),bbox_inches='tight')
    #plt.show()
    plt.close(fig)
os.chdir(dir0)
#6839_56425_781.pdf

['z_hits',
 'peak_id',
 'fakesn',
 'mjd_hits',
 'peak_sn',
 'fiber_hits',
 'peak_wave',
 'plate_hits',
 'spec_id']

In [9]:
lya_hits['peak_wave'].shape

(921,)

In [4]:
t.info

<Table length=728>
  name  dtype
------- -----
id@json int64
  plate int64
    mjd int64
  fiber int64
    tag  str6
   note str35

In [3]:
plate_list_pickle

array([10243, 10272, 10274, ...,  9631,  9631,  9632])

In [7]:
plate_hits = np.array(t['plate'])